In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
from sequence import sequence_TTL
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load bitstream with custom overlay
soc = QickSoc()
# Since we're running locally on the QICK, we don't need a separate QickConfig object.
# If running remotely, you could generate a QickConfig from the QickSoc:
#     soccfg = QickConfig(soc.get_cfg())
# or save the config to file, and load it later:
#     with open("qick_config.json", "w") as f:
#         f.write(soc.dump_cfg())
#     soccfg = QickConfig("qick_config.json")
soccfg = soc
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Global clocks (MHz): tProcessor 350.000, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 0, envelope memory 65536 samples
		DAC tile 0, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz
	1:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz

	2 readout channels:
	0:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 0, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 4, tProc input 0
	1:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 1, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 5, tProc input 1

	2 DACs:
		DAC tile 0, ch 0 is DAC_B
		DAC tile 2, ch 0 is DAC_A

	2 ADCs:
		ADC tile 0, ch 0 is ADC_D
		ADC tile 0, ch 1 is ADC_C

	4 digital output pins (tProc output 7):
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3

In [12]:
def divide_nested_list(nested_list, divisor):
    result = []
    for sublist in nested_list:
        divided_sublist = []
        for number in sublist:
            divided_sublist.append(round(number / divisor))
        result.append(divided_sublist)
    return result

In [13]:
# andrin: I was just hacking this in quickly to have four pulses

class QuadPulseProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

#         set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
#         configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        gain = cfg["pulse_gain"]

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
                                         )
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
                                         waveform="measure")
        elif style == "arb":
            self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
                                         waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.cfg["time"] = divide_nested_list(self.cfg["time"],2.6)
        self.cfg["length"] = divide_nested_list(self.cfg["length"],2.6)
        
        time_list,length,pins_seq = sequence_TTL(self.cfg["time"], self.cfg["length"], self.cfg["pins"])
        for time_index, time in enumerate(time_list):
            self.trigger(adcs=self.ro_chs,
                    pins=pins_seq[time_index], 
                    adc_trig_offset=self.cfg["adc_trig_offset"],
                     t = time,
                    width = length[time_index])
        self.wait_all()
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

config={"res_ch":0, # --Fixed
        "ro_chs":[0], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        "readout_length":400, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks
        "pulse_gain": 3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units
        "pulse_freq": 1003, # [MHz]
        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks
        "soft_avgs":100,

        "length":[[200,200,200,200],[200,200,200,200],[200,200,200,200],[200,200,200,200]], # [Clock ticks]
        "pins":[1,2,0,3],
        "time":[[0,400,800,1200],[0,400,800,1200],[0,400,800,1200],[0,400,800,1200]],
        
        "pulse_num": 4
        # Try varying soft_avgs from 1 to 200 averages
       }

prog = QuadPulseProgram(soccfg, config)
# print(prog.acquire(soc))
# print(prog.di_buf)
import time
while True:
    iq_list = prog.acquire_decimated(soc)
    time.sleep(2)
# iq_list = prog.acquire_decimated(soc)

plt.figure(1)
for ii, iq in enumerate(iq_list):
#     plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
#     plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

Readout time 154 appears to conflict with previous readout ending at 384.830729?
Readout time 308 appears to conflict with previous readout ending at 538.830729?
Readout time 462 appears to conflict with previous readout ending at 692.830729?


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
# # andrin: I was just hacking this in quickly to have four pulses

# class QuadPulseProgram(AveragerProgram):
#     def initialize(self):
#         cfg=self.cfg   
#         res_ch = cfg["res_ch"]

# #         set the nyquist zone
#         self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
# #         configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
#         for ch in cfg["ro_chs"]:
#             self.declare_readout(ch=ch, length=self.cfg["readout_length"],
#                                  freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

#         # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
#         freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
#         gain = cfg["pulse_gain"]

#         style=self.cfg["pulse_style"]

#         if style in ["flat_top","arb"]:
#             sigma = cfg["sigma"]
#             self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
#         if style == "const":
#             self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
#                                          )
#         elif style == "flat_top":
#             # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
#             self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
#                                          waveform="measure")
#         elif style == "arb":
#             self.default_pulse_registers(ch=res_ch, style=style, freq=freq, gain=gain, 
#                                          waveform="measure")
        
#         self.synci(200)  # give processor some time to configure pulses
    
#     def body(self):
#         self.trigger(adcs=self.ro_chs, pins=[0], adc_trig_offset=100, t = 0, width = 150)
#         self.trigger(adcs=self.ro_chs, pins=[1], adc_trig_offset=100, t = 50, width = 350)
#         self.trigger(adcs=self.ro_chs, pins=[1], adc_trig_offset=100, t = 250, width = 200)
#         self.trigger(adcs=self.ro_chs, pins=[1], adc_trig_offset=100, t = 850, width = 200)
#         self.trigger(adcs=self.ro_chs, pins=[1], adc_trig_offset=100, t = 1250, width = 200)
        
#         self.wait_all()
#         self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

# config={"res_ch":0, # --Fixed
#         "ro_chs":[0], # --Fixed
#         "reps":1, # --Fixed
#         "relax_delay":1.0, # --us
#         "res_phase":0, # --degrees
#         "pulse_style": "const", # --Fixed
#         "readout_length":400, # [Clock ticks]
#         # Try varying readout_length from 50-1000 clock ticks
#         "pulse_gain": 3000, # [DAC units]
#         # Try varying pulse_gain from 500 to 30000 DAC units
#         "pulse_freq": 1003, # [MHz]
#         "adc_trig_offset": 100, # [Clock ticks]
#         # Try varying adc_trig_offset from 100 to 220 clock ticks
#         "soft_avgs":100,

#         "length":[[200,200,200,200],[200,200,200,200],[200,200,200,200],[200,200,200,200]], # [Clock ticks]
#         "pins":[0,1,2,3],
#         "time":[0,400,800,1200],
# #         "time":[[0,400,800,1200],[0,400,800,1200],[0,400,800,1200],[0,400,800,1200]],
        
#         "pulse_num": 4
#         # Try varying soft_avgs from 1 to 200 averages
#        }

# prog = QuadPulseProgram(soccfg, config)
# # print(prog.acquire(soc))
# # print(prog.di_buf)
# import time
# while True:
#     iq_list = prog.acquire_decimated(soc)
#     time.sleep(2)
# # iq_list = prog.acquire_decimated(soc)

# plt.figure(1)
# for ii, iq in enumerate(iq_list):
# #     plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
# #     plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
#     plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
# plt.ylabel("a.u.")
# plt.xlabel("Clock ticks")
# plt.title("Averages = " + str(config["soft_avgs"]))
# plt.legend()

Readout time 50 appears to conflict with previous readout ending at 384.830729?
Readout time 250 appears to conflict with previous readout ending at 434.830729?


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 